<a href="https://colab.research.google.com/github/shuxiny9/BreederSystem/blob/main/langchain_chatbot_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain Chatbot Tutorial 🤖

---


Welcome to this beginner-friendly tutorial on building a chatbot with LangChain and LangGraph!

## What you'll learn:
- How to set up a simple AI chatbot
- Understanding the basic components of LangChain/LangGraph
- How to interact with Google's Gemini AI model
- Building a conversational flow

## Prerequisites:
- No prior Python knowledge required! We'll explain everything step by step
- A Google API key (we'll show you how to get one)

Let's get started! 🚀


In [ ]:
# Install the required packages
# This might take a few minutes the first time you run it

%pip install langchain langgraph langchain-google-genai langchain-tavily

print("✅ Installation complete! Ready to build our chatbot.")


In [ ]:
import os

# Replace "your_api_key_here" with your actual Google API key
# Make sure to keep the quotes around it!
from google.colab import userdata


# os.environ["GOOGLE_API_KEY"] = "your_api_key_here"
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")


# This line checks if the key was set correctly
if os.environ.get("GOOGLE_API_KEY") == "your_api_key_here":
    print("⚠️  Don't forget to replace 'your_api_key_here' with your actual API key!")
else:
    print("✅ API key is set up!")


✅ API key is set up!


In [ ]:
# Import statements - these bring in the tools we need
from typing import Annotated                      # Helps with type hints (optional but good practice)
from typing_extensions import TypedDict           # Helps us define data structures

from langgraph.graph import StateGraph, START, END    # The main graph components
from langgraph.graph.message import add_messages      # Handles message management
from langchain.chat_models import init_chat_model     # Connects to AI models

print("✅ All tools imported successfully!")


✅ All tools imported successfully!


In [ ]:
# Define what our chatbot will remember
class State(TypedDict):
    # This stores all the messages in our conversation
    # The `add_messages` part tells LangGraph to add new messages to the list
    # instead of replacing the whole list
    messages: Annotated[list, add_messages]

print("✅ State structure defined!")
print("Our chatbot will remember: messages in the conversation")


✅ State structure defined!
Our chatbot will remember: messages in the conversation


In [ ]:
# Connect to Google's Gemini AI model
# "gemini-2.0-flash" is a fast and capable version of Google's AI
llm = init_chat_model("google_genai:gemini-2.0-flash")

print("✅ Connected to Gemini AI!")
print("Model: gemini-2.0-flash")


✅ Connected to Gemini AI!
Model: gemini-2.0-flash


In [ ]:
# This function is the "brain" of our chatbot
def chatbot(state: State):
    """
    This function takes the conversation history and generates a response.

    How it works:
    1. Takes all the messages from the conversation so far
    2. Sends them to the AI model (Gemini)
    3. Gets back a response
    4. Returns the response in the format LangGraph expects
    """
    response = llm.invoke(state["messages"])
    return {"messages": [response]}

print("✅ Chatbot function created!")
print("This function will process messages and generate AI responses.")


✅ Chatbot function created!
This function will process messages and generate AI responses.


In [ ]:
# Step 1: Create a graph builder
graph_builder = StateGraph(State)

# Step 2: Add our chatbot function as a "node" in the graph
# The first part ("chatbot") is just a name we give it
# The second part (chatbot) is our actual function
graph_builder.add_node("chatbot", chatbot)

# Step 3: Define the flow: START → chatbot → END
graph_builder.add_edge(START, "chatbot")  # When conversation starts, go to chatbot
graph_builder.add_edge("chatbot", END)    # After chatbot responds, end this turn

# Step 4: Build the final graph
graph = graph_builder.compile()

print("✅ Conversation graph built!")
print("Flow: START → chatbot function → END")


✅ Conversation graph built!
Flow: START → chatbot function → END


In [ ]:
# Function to send a message and get a response
def send_message(user_input: str):
    """
    Send a message to the chatbot and display the response.
    """
    print(f"User: {user_input}")

    # Send the message through our graph
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            response = value["messages"][-1].content
            print(f"Assistant: {response}")
    print("-" * 50)  # Separator line

# Test with a simple message
send_message("Why should I become an AI Engineer today")


User: Why should I become an AI Engineer today
Assistant: Becoming an AI Engineer today can be a really smart career move for several reasons. Here's a breakdown of the key benefits:

**1. High Demand and Job Security:**

*   **AI is booming:**  Artificial intelligence is no longer a futuristic concept. It's rapidly being integrated into almost every industry, from healthcare and finance to transportation and entertainment. This widespread adoption fuels a massive demand for skilled AI professionals.
*   **Shortage of talent:** The demand for AI engineers far outweighs the supply.  This means you'll have excellent job security and a high likelihood of finding a position that aligns with your interests and skills.
*   **Future-proof career:** AI is a rapidly evolving field, but the underlying principles and the need for professionals who can build and maintain AI systems are likely to remain strong for the foreseeable future.

**2. Lucrative Compensation:**

*   **High earning potential

In [ ]:
# Example 1: Ask about programming
send_message("Can you explain what Python is in simple terms?")


In [ ]:
# Example 2: Ask for help with something
send_message("I'm new to AI and chatbots. What should I learn next?")


In [ ]:
# Example 3: Try a creative question
send_message("Write a short poem about robots learning to code")


In [ ]:
# Interactive chat session
print("🤖 Welcome to your chatbot!")
print("Type your messages below. Type 'quit' to exit.")
print("=" * 50)

try:
    while True:
        # Get user input
        user_input = input("\nYou: ")

        # Check if user wants to quit
        if user_input.lower() in ["quit", "exit", "q", "bye"]:
            print("Chatbot: Goodbye! Thanks for chatting! 👋")
            break

        # Send message to chatbot
        print("Chatbot: ", end="")
        for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
            for value in event.values():
                response = value["messages"][-1].content
                print(response)

except KeyboardInterrupt:
    print("\n\nChatbot: Goodbye! Thanks for chatting! 👋")
except:
    print("\n\nNote: Interactive input might not work in all environments.")
    print("If you see this message, try using the send_message() function instead!")


🤖 Welcome to your chatbot!
Type your messages below. Type 'quit' to exit.

You: Tell me about CodePath
Chatbot: CodePath is a non-profit organization that aims to bridge the gap between computer science education in universities and the skills required for success in the tech industry. They do this by providing free, intensive courses and workshops to college students, focusing on practical skills and career development.

Here's a breakdown of key aspects of CodePath:

**Mission and Goals:**

*   **Close the skills gap:** Equip students with in-demand tech skills that are often missing from traditional computer science curricula.
*   **Increase diversity in tech:** Focus on reaching students from underrepresented backgrounds, including women, students of color, and students from low-income families.
*   **Improve career readiness:** Provide students with the tools and resources they need to succeed in tech interviews and land internships/full-time jobs.
*   **Partner with universities: